In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json

json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'
json_path_onedrive = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\'

### 1. Extract Stimulation Parameters

In [ ]:
json_files_to_match = pd.read_excel(os.path.join(
    json_path_onedrive, 'results', 'Stim_pars', 'Done',
    'Jsons_12mfu_StimPars_ToExtract.xlsx'
))

#json_files_to_match = json_files_to_match.drop(2)
json_files_to_match

In [ ]:
import pickle
reload(extract_features)
saving_path = os.path.join(
    json_path_onedrive, 'results', 'Stim_pars', 'Done'
)

#for index, row in json_files_to_match.iterrows():

index = 26 
json_12mfu_stimPars = os.path.join(
    json_path_project, 
    json_files_to_match.loc[index,'SubID'],
    json_files_to_match.loc[index,'Con_reason'],
    json_files_to_match.loc[index,'json_fileName']
)

with open(json_12mfu_stimPars) as file:
            # Load the JSON data
            data = json.load(file)

ElectrodeType = str(json_files_to_match.loc[index,'ElectrodeType'])

stim_pars_dict = extract_features.extract_StimPars(data, ElectrodeType)

subID = json_files_to_match.loc[index,'SubID']
saving_json_name = f'{subID}_StimPars_12mfu.pkl'

with open(os.path.join(
            saving_path,
            saving_json_name
        ), "wb") as file:
            pickle.dump(stim_pars_dict, file)

### 2. Calculate TEED

In [ ]:
stims_path = os.path.join(
    json_path_onedrive, 'results', 'Stim_pars', 'Done'
)

stim_pars_FileName = "Sub047_StimPars_12mfu.pkl"

with open(os.path.join(stims_path,
    stim_pars_FileName), "rb") as file:
    stim_dat = pickle.load(file)

subID = stim_pars_FileName.split('_')[0]

In [ ]:
reload(extract_features)

ElectrodeType = 'SenSight'
voltage_L, TEED_L, voltage_R, TEED_R = extract_features.calculate_TEDD(stim_dat, ElectrodeType)

In [ ]:
df = pd.DataFrame(stim_dat)
df.insert(0, 'SubID', subID)

df['Voltage'] = [voltage_L, voltage_R]
df['TEDD'] = [TEED_L, TEED_R]
df

In [ ]:
df.to_csv(
    os.path.join(
        json_path_onedrive,
        'results',
        'Stim_pars',
        'TEED',
        f'{subID}_TEDD.csv'
    ), index=False)  # Specify the desired file name
